In [2]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from ipywidgets import interact

nibabel raw file reference

In [3]:
# define path
base_path = '../data/BRaTS2021/BRaTS2021_raw/' 
sample_id = '00002'

path = base_path  + 'BraTS2021_' + sample_id  + '/BraTS2021_' + sample_id + '_flair.nii.gz'

print(path)

flair   = nib.load(base_path  + 'BraTS2021_' + sample_id  + '/BraTS2021_' + sample_id + '_flair.nii.gz')
seg     = nib.load(base_path  + 'BraTS2021_' + sample_id  + '/BraTS2021_' + sample_id + '_seg.nii.gz')
T1      = nib.load(base_path  + 'BraTS2021_' + sample_id  + '/BraTS2021_' + sample_id + '_t1.nii.gz')
T1ce    = nib.load(base_path  + 'BraTS2021_' + sample_id  + '/BraTS2021_' + sample_id + '_t1ce.nii.gz')
T2      = nib.load(base_path  + 'BraTS2021_' + sample_id  + '/BraTS2021_' + sample_id + '_t2.nii.gz')

# print number of slices
imgshape = flair.shape
print(f"Image resolution: {imgshape[0]}x{imgshape[1]}")
print(f"Number of slices: {imgshape[2]}")

../data/BRaTS2021/BRaTS2021_raw/BraTS2021_00002/BraTS2021_00002_flair.nii.gz
Image resolution: 240x240
Number of slices: 155


In [21]:
flair_data = flair.get_fdata()

# print image datatype
print(f"Image datatype: {flair_data.dtype}")
print(type(flair_data))

Image datatype: float64
<class 'numpy.ndarray'>


Modalities and tumour classes

In [22]:
classes_dict = {
    0 : 'B/W = healthy',
    1 : 'Red = necrotic',
    2 : 'Green = edematous',
    4 : 'Blue = enhancing'
}

modalities = ["flair", "t1", "t1ce", "t2", "seg"]

In [23]:

def visualize_3d_labels(layer):
    # change colours of segmentation result    
    mask = seg.get_fdata()
    color_segmentation = np.zeros((240, 240, 3), dtype=int)
    gray_segmentation = mask[:, :, layer]
    print( np.unique(gray_segmentation).astype(int))

    # print segmentation result
    print([classes_dict[result] for result in np.unique(gray_segmentation).astype(int)])
    # print(f"Segmentation result: {[classes_dict[result] for result in np.unique(gray_segmentation)]}")

                                                                # Black (healthy tissue) = 0
    color_segmentation[gray_segmentation == 1] = [255, 0, 0]    # Red (necrotic tumor core) = 1
    color_segmentation[gray_segmentation == 2] = [0, 255,0]     # Green (peritumoral edematous/invaded tissue) = 2
    color_segmentation[gray_segmentation == 4] = [0, 0, 255]    # Blue (enhancing tumor) = 4
 
    plt.figure(figsize=(10, 5))
    
    plt.imshow(flair_data[:, :, layer], cmap='gray');
    # overlay mask
    plt.imshow(color_segmentation, cmap='gray', alpha=0.6)
    plt.axis('off')
    plt.tight_layout()
    return layer

# interact(visualize_3d_labels, layer=(0, flair_data.shape[2] - 1));

# in case this does not work, see: https://stackoverflow.com/questions/36351109/ipython-notebook-ipywidgets-does-not-show

rgb2gray conversion

In [24]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

Pre-processed data, loaded from nii individual slices

In [25]:
# define path (BraTS2021_00610_flair_5)
base_path = '../data/BRaTS2021/BRaTS2021_processed_nii/' 
split = 'test'
base_path = os.path.join(base_path, split)

def visualize_3d_labels(layer):
    # change colours of segmentation result    
    # mask = seg.get_fdata()
    color_segmentation = np.zeros((240, 240, 3), dtype=int)
    # gray_segmentation = mask[:, :, layer]

    img_path = f'{base_path}/BraTS2021_{sample_id}_seg_{layer}.nii.gz'
    print(img_path)

    # read image
    img_nii = nib.load(img_path)
    gray_segmentation = img_nii.get_fdata()
    print( np.unique(gray_segmentation).astype(int))
    print("gray_segmentation shape: ", np.shape(gray_segmentation))
            

    # print segmentation result
    print([classes_dict[result] for result in np.unique(gray_segmentation).astype(int)])
    # print(f"Segmentation result: {[classes_dict[result] for result in np.unique(gray_segmentation)]}")

                                                                # Black (healthy tissue) = 0
    color_segmentation[gray_segmentation == 1] = [255, 0, 0]    # Red (necrotic tumor core) = 1
    color_segmentation[gray_segmentation == 2] = [0, 255,0]     # Green (peritumoral edematous/invaded tissue) = 2
    color_segmentation[gray_segmentation == 4] = [0, 0, 255]    # Blue (enhancing tumor) = 4
 
    plt.figure(figsize=(10, 5))
    
    plt.imshow(flair_data[:, :, layer], cmap='gray');
    # overlay mask
    plt.imshow(color_segmentation, cmap='gray', alpha=0.6)
    plt.axis('off')
    plt.tight_layout()
    return layer

# interact(visualize_3d_labels, layer=(0, flair_data.shape[2] - 1));


Calculate mean and variance for each modality

In [38]:
# Get the list of all the files in the directory
base_path = '../data/BRaTS2021/BRaTS2021_processed_nii/test'

# modalities
# mri_modalities = ["flair", "T1", "T1ce", "T2"]
mri_modalities = ["seg"]

# cases
slices_per_case = 155
files = sorted(os.listdir(base_path))
files_per_case = slices_per_case * (len(modalities) + 1)
n_cases = len(files) // files_per_case # 775
print("Number of files in folder: ", len(files))

# get the list of all the cases (example=BraTS2021_00021_flair_0) we only need 'BraTS2021_xxxxx'
cases = [files[i][:15] for i in range(0, n_cases*files_per_case, files_per_case)]
print("Number of cases: ", n_cases)

# calculate mean and variance of each modality 
# filename = BraTS2021_00000_flair_0.nii.gz
def calculate_mean_and_variance(filename):
    img = nib.load(filename)
    img_data = img.get_fdata()
    mean = np.mean(img_data)
    variance = np.var(img_data)
    print(f"Mean of {modality}: {mean}")
    print(f"Variance of {modality}: {variance}")

sum_health = 0
# sum_non_health = 0
sum_enhancing = 0
sum_edematous = 0
sum_necrotic = 0 

for modality in mri_modalities:
    for case in cases:
        for slice_idx in range(0, 154):
            filename = case + '_'  + modality + '_' + str(slice_idx) + '.nii.gz'
            # calculate_mean_and_variance(os.path.join(base_path, filename))

            # count number of class labels for 0, 1, 2, 4
            if modality == "seg":
                img = nib.load(os.path.join(base_path, filename))
                img_data = img.get_fdata()

                # measure the number of pixels in each class
                sum_health += np.sum(img_data == 0)
                sum_enhancing += np.sum(img_data == 4)
                sum_edematous += np.sum(img_data == 2)
                sum_necrotic += np.sum(img_data == 1)

# percentage of each class
total = sum_health + sum_edematous + sum_enhancing + sum_necrotic
print(f"Percentage of healthy: {sum_health / total * 100 :2f} %")
print(f"Percentage of edematous: {sum_edematous / total * 100 :.3f} %")
print(f"Percentage of enhancing: {sum_enhancing / total * 100 :.3f} %")
print(f"Percentage of necrotic: {sum_necrotic / total * 100 :.3f} %")


Number of files in folder:  96875
Number of cases:  104
Percentage of healthy: 98.796930 %
Percentage of edematous: 0.767 %
Percentage of enhancing: 0.266 %
Percentage of necrotic: 0.171 %


Tensor board test

In [27]:
# import tensorboardX
%load_ext tensorboard
%tensorboard --logdir 'saved/log/BraTS2021_Base_Unet/0530_165145'